<a href="https://colab.research.google.com/github/seongheechoi/education/blob/main/%EC%8B%A4%EC%8A%B5_3_1_optimizing_operators_with_schedule_templates_and_AutoTVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TVM 실습자료 3.1: Optimizing Operators with Schedule Templates and AutoTVM**

In [ ]:
!pip install numpy==1.26.4
import numpy as np
print(np.__version__)
!pip list | grep numpy

1.26.4
numpy                                 1.26.4


In [ ]:
# Linux/MacOS CPU build only!
# See tlcpack.ai for other pre-built binaries including CUDA
!python -m pip install --upgrade pip
!pip install apache-tvm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 56.4 MB/s eta 0:00:00


**Install dependencies**

In [ ]:
!pip3 install --user psutil xgboost cloudpickle

In [ ]:
import logging
import sys

import numpy as np
import tvm
from tvm import te
import tvm.testing

# the module is called `autotvm`
from tvm import autotvm

**Basic Matrix Multiplication with TE**

In [ ]:
def matmul_basic(N, L, M, dtype):

    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    yo, yi = s[C].split(y, 8)
    xo, xi = s[C].split(x, 8)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

**A Basic Matrix Multiplication Template**

In [ ]:
# Matmul V1: List candidate values
@autotvm.template("tutorial/matmul_v1")  # 1. use a decorator
def matmul_v1(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    # 2. get the config object
    cfg = autotvm.get_config()

    # 3. define search space
    cfg.define_knob("tile_y", [1, 2, 4, 8, 16])
    cfg.define_knob("tile_x", [1, 2, 4, 8, 16])

    # 4. schedule according to config
    yo, yi = s[C].split(y, cfg["tile_y"].val)
    xo, xi = s[C].split(x, cfg["tile_x"].val)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

**A Matrix Multiplication Template with the Advanced Parameter API**

In [ ]:
@autotvm.template("tutorial/matmul")
def matmul(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # schedule
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    ##### define space begin #####
    cfg = autotvm.get_config()
    cfg.define_split("tile_y", y, num_outputs=2)
    cfg.define_split("tile_x", x, num_outputs=2)
    ##### define space end #####

    # schedule according to config
    yo, yi = cfg["tile_y"].apply(s, C, y)
    xo, xi = cfg["tile_x"].apply(s, C, x)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

**Auto-tuners in TVM**

In [ ]:
N, L, M = 512, 512, 512
task = autotvm.task.create("tutorial/matmul", args=(N, L, M, "float32"), target="llvm")
print(task.config_space)

ConfigSpace (len=100, range_length=100, space_map=
   0 tile_y: Split(policy=factors, product=512, num_outputs=2) len=10
   1 tile_x: Split(policy=factors, product=512, num_outputs=2) len=10
)


In [ ]:
# logging config (for printing tuning log to the screen)
logging.getLogger("autotvm").setLevel(logging.DEBUG)
logging.getLogger("autotvm").addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
measure_option = autotvm.measure_option(builder="local", runner=autotvm.LocalRunner(number=5))

# Begin tuning with RandomTuner, log records to file `matmul.log`
# You can use alternatives like XGBTuner.
tuner = autotvm.tuner.RandomTuner(task)
tuner.tune(
    n_trial=10,
    measure_option=measure_option,
    callbacks=[autotvm.callback.log_to_file("matmul.log")],
)

waiting for device...


DEBUG:autotvm:waiting for device...


device available


DEBUG:autotvm:device available


Get devices for measurement successfully!


...

DEBUG:autotvm:No: 8	GFLOPS: 10.84/11.58	result: MeasureResult(costs=(0.024761730399999998,), error_no=0, all_cost=0.8073360919952393, timestamp=1752211679.9346428)	[('tile_y', [-1, 32]), ('tile_x', [-1, 256])],None,85


No: 9	GFLOPS: 0.77/11.58	result: MeasureResult(costs=(0.3473268688,), error_no=0, all_cost=5.884622573852539, timestamp=1752211686.1256928)	[('tile_y', [-1, 32]), ('tile_x', [-1, 2])],None,15


DEBUG:autotvm:No: 9	GFLOPS: 0.77/11.58	result: MeasureResult(costs=(0.3473268688,), error_no=0, all_cost=5.884622573852539, timestamp=1752211686.1256928)	[('tile_y', [-1, 32]), ('tile_x', [-1, 2])],None,15


No: 10	GFLOPS: 9.57/11.58	result: MeasureResult(costs=(0.028058876600000004,), error_no=0, all_cost=0.9659364223480225, timestamp=1752211686.8450854)	[('tile_y', [-1, 256]), ('tile_x', [-1, 128])],None,78


DEBUG:autotvm:No: 10	GFLOPS: 9.57/11.58	result: MeasureResult(costs=(0.028058876600000004,), error_no=0, all_cost=0.9659364223480225, timestamp=1752211686.8450854)	[('tile_y', [-1, 256]), ('tile_x', [-1, 128])],None,78


In [ ]:
# apply history best from log file
with autotvm.apply_history_best("matmul.log"):
    with tvm.target.Target("llvm"):
        s, arg_bufs = matmul(N, L, M, "float32")
        func = tvm.build(s, arg_bufs)

# check correctness
a_np = np.random.uniform(size=(N, L)).astype(np.float32)
b_np = np.random.uniform(size=(L, M)).astype(np.float32)
c_np = a_np.dot(b_np)

c_tvm = tvm.nd.empty(c_np.shape)
func(tvm.nd.array(a_np), tvm.nd.array(b_np), c_tvm)

tvm.testing.assert_allclose(c_np, c_tvm.numpy(), rtol=1e-4)

Finish loading 10 records


DEBUG:autotvm:Finish loading 10 records


In [ ]:
print(tvm.lower(s, arg_bufs, simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((512, 512), "float32"), B: T.Buffer((512, 512), "float32"), C: T.Buffer((512, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for i_outer, j_outer in T.grid(16, 8):
            C_1 = T.Buffer((262144,), data=C.data)
            for i_inner_init, j_inner_init in T.grid(32, 64):
                C_1[i_outer * 16384 + i_inner_init * 512 + j_outer * 64 + j_inner_init] = T.float32(0)
            for k, i_inner, j_inner in T.grid(512, 32, 64):
                cse_var_3: T.int32 = j_outer * 64
                cse_var_2: T.int32 = i_outer * 16384 + i_inner * 512
                cse_var_1: T.int32 = cse_var_2 + cse_var_3 + j_inner
                A_1 = T.Buffer((262144,), data=A.data)
                B_1 = T.Buffer((262144,), data=B.data)
                C_1[cse_var_1] = C_1[cse_var_1] + A_1